In [5]:
#Read Table Method
import pandas as pd
import cx_Oracle
import sqlalchemy

#establish connection -- USE SPECIFIC PATH
cx_Oracle.init_oracle_client(lib_dir=r"/Users/ethanblumenfeld/Downloads/instantclient_19_8")

In [6]:
def readTable(user, password, dsn, table):


    connection = cx_Oracle.connect(user=user, password=password, dsn=dsn)

    cursor = connection.cursor()

    # return table as Data Frame

    query = """select * 
            from """ + table;
    df = pd.read_sql(query, con=connection)

    #print(df)
    return df
   


In [20]:
#example
df = readTable("admin", "!1Iloveoracle", "db202203241240_high", "team_sample")
print(df)

   TEAM_NAME  GAMEDATE   STAT1      STAT2  STAT3 STAT4 STAT5
0      team1  4/7/2022     one        two  three  four  five
1      team4  4/7/2022    good        bad   good   bad  good
2      team5  4/7/2022    good        bad   good   bad  good
3      team5  4/7/2022  great!  Terrible!   None  None  None
4      team5  4/7/2022    good        bad   good   bad  good
5      team5  4/7/2022  great!        bad   good   bad  good
6      team2  4/7/2022     one        two  three  four  five
7      team3  4/7/2022     one        two  three  four  five
8      team4  4/7/2022    good        bad   good   bad  good
9      team5  4/7/2022    good        bad   good   bad  good
10      None      None    None       None   None  None  None


In [41]:
# valDict is a dictionary of table column name mapped to value.
def writeToTable(user, password, dsn, table, valDict):


    connection = cx_Oracle.connect(user=user, password=password, dsn=dsn)

    cursor = connection.cursor()
    defaultDict = {'Predictive_Outcome':0}
    
    values = []
    for column in defaultDict:
        if column in valDict:
            values.append(valDict[column])
        else:
            values.append(defaultDict[column])
            
    cursor.execute("insert into "+table+"(Predictive_Outcome) values (:1)", values)

    connection.commit()

#example
#values = {'TEAM_NAME':'team5' , 'GAMEDATE':'4/7/2022' , 'STAT1':'great!' , 'STAT2':'bad' , 'STAT3':'good' , 'STAT4':'bad', 'STAT5':'good'}
#values = {'TEAM_NAME':'team5' , 'GAMEDATE':'4/7/2022' , 'STAT1':'great!' , 'STAT2':'Terrible!'}
#writeToTable("admin", "!1Iloveoracle", "db202203241240_high", "team_sample", values)

Method 1:

In [25]:
%%time
df = readTable("NBACOVID", "!1Iloveoracle", "db202203241240_high", "DATASET")

CPU times: user 75.5 ms, sys: 8.87 ms, total: 84.3 ms
Wall time: 2.19 s


Method 2:

In [21]:
import timeit
start_time = timeit.default_timer()

#Code to time
df = readTable("NBACOVID", "!1Iloveoracle", "db202203241240_high", "DATASET")

elapsed = timeit.default_timer() - start_time
print(elapsed, "Seconds")

2.902796809999927 Seconds


In [43]:
import timeit
start_time = timeit.default_timer()

#Code to time
values = {'Predictive_Outcome':5}
writeToTable("NBACOVID", "!1Iloveoracle", "db202203241240_high", "Predictions", values)

elapsed = timeit.default_timer() - start_time
print(elapsed, "Seconds")

0.9663302169992676 Seconds


Method 1 for XGBoost:

In [72]:
#Code to time First XGBoost model
import timeit
start_time = timeit.default_timer()

# load data
df = readTable("NBACOVID", "!1Iloveoracle", "db202203241240_high", "PYTABCHHUN")

elapsed_read = timeit.default_timer() - start_time
print("Read:", elapsed_read, "Seconds")

import xgboost
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# split data into X and y
X = df.iloc[:,1:]
Y = df['Outcome']
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

elapsed = timeit.default_timer() - start_time - elapsed_read
print("Inference:", elapsed, "Seconds")

for predic in predictions[:100]:
    values = {'Predictive_Outcome':int(predic)}
    writeToTable("NBACOVID", "!1Iloveoracle", "db202203241240_high", "Predictions", values)
    
elapsed = timeit.default_timer() - start_time - elapsed - elapsed_read
print("Write:", elapsed, "Seconds")
    
elapsed = timeit.default_timer() - start_time
print("Total:",elapsed, "Seconds")

Read: 2.5139523960006045 Seconds
Accuracy: 87.52%
Inference: 1.3611832810001943 Seconds
Write: 95.11403420299939 Seconds
Total: 98.98941843800094 Seconds


In [73]:
#Code to time logistic Regression model
import timeit
print("Model: Logistic Regression")
start_time = timeit.default_timer()

# load data
df = readTable("NBACOVID", "!1Iloveoracle", "db202203241240_high", "PYTABCHHUN")

elapsed_read = timeit.default_timer() - start_time
print("Read:", elapsed_read, "Seconds")

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# split data into X and y
X = df.iloc[:,1:]
Y = df['Outcome']
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# fit model 
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

score = model.score(X_test, y_test)
print(score)

elapsed = timeit.default_timer() - start_time - elapsed_read
print("Inference:", elapsed, "Seconds")

for predic in y_pred[:100]:
    values = {'Predictive_Outcome':int(predic)}
    writeToTable("NBACOVID", "!1Iloveoracle", "db202203241240_high", "Predictions", values)
    
elapsed = timeit.default_timer() - start_time - elapsed - elapsed_read
print("Write:", elapsed, "Seconds")
    
elapsed = timeit.default_timer() - start_time
print("Total:",elapsed, "Seconds")

Model: Logistic Regression
Read: 4.2152628429994365 Seconds
0.8751793400286944
Inference: 5.654190036000728 Seconds
Write: 135.59037580199947 Seconds
Total: 145.46005009999863 Seconds


In [75]:
#Code to time Random Forrest Classification model
import timeit
print("Model: Random Forrest Classification")
start_time = timeit.default_timer()

# load data
df = readTable("NBACOVID", "!1Iloveoracle", "db202203241240_high", "PYTABCHHUN")

elapsed_read = timeit.default_timer() - start_time
print("Read:", elapsed_read, "Seconds")

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.model_selection import train_test_split


# split data into X and y
X = df.iloc[:,1:]
Y = df['Outcome']
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# fit model 
model = RandomForestClassifier()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)


elapsed = timeit.default_timer() - start_time - elapsed_read
print("Inference:", elapsed, "Seconds")

for predic in y_pred[:100]:
    values = {'Predictive_Outcome':int(predic)}
    writeToTable("NBACOVID", "!1Iloveoracle", "db202203241240_high", "Predictions", values)
    
elapsed = timeit.default_timer() - start_time - elapsed - elapsed_read
print("Write:", elapsed, "Seconds")
    
elapsed = timeit.default_timer() - start_time
print("Total:",elapsed, "Seconds")

Model: Random Forrest Classification
Read: 2.0875794740004494 Seconds
Inference: 0.7582120250008302 Seconds
Write: 113.74644476899994 Seconds
Total: 116.59257590200104 Seconds
